# **Exploration des données - DynamicEarthNet**
## **1. Présentation du contexte et du jeu de données**

Dans ce projet, l’objectif principal est la réalisation d’une **segmentation sémantique spatiale** à partir d’images satellites issues de la base de données **DynamicEarthNet**. Cette dernière se compose d’images satellites multispectrales (RGB + proche infrarouge) annotées manuellement pour différents types de couverture terrestre.

Le jeu de données se caractérise par :
- **Résolution spatiale** : 3 mètres par pixel (chaque image fait 1024x1024 pixels).
- **Annotations manuelles précises**, pixel par pixel, classifiant la couverture terrestre en plusieurs classes sémantiques clairement définies.

> Source : *DynamicEarthNet Paper, Toker et al. 2022*.

---



La première étape de notre projet à donc été de visualiser et de prendre en main la données. Pour cela deux module ont été crée : [sat_image_reader](src/sat_image_reader.py) et [classes_reader](src/classes_reader.py).

Grâçe à cela, nous avons pu analyser diverses images et les prendre en main dans un [notebook](exploration_données_Alexandre.ipynb).
Nous avons alors pu comprendre les diverses classes de DynamicEarthNet, leur implémentation (sous 8 bandes à valeurs 0 ou 255 si la classe est présente) et l'évolution temporelle.


## **2. Classes sémantiques utilisées dans l'exploration**

Le jeu de données DynamicEarthNet comporte au total **7 classes sémantiques** différentes :

1. **Impervious Surface** – Surfaces imperméables (routes, bâtiments).
2. **Forest & Other Vegetation** – Forêt et autres végétations.
3. **Soil** – Sols nus ou terres agricoles non cultivées.
4. **Agriculture** – Zones agricoles cultivées.
5. **Wetlands** – Zones humides.
6. **Water** – Plans d’eau.
7. **Snow & Ice** – Neige et glace (rare).

Mais elles ne sont pas forcèment toutes présente à chaque fois : par exemple dans une zone étudiée du notebook (**10N-123W-45N**), seules **3 classes principales** sont présentes :

- **Impervious Surface (bande 1)**
- **Forest and Other Vegetation (bande 3)**
- **Soil (bande 5)**

---


Nous avons donc pris cela en compte lors de la création de notre [dataloader](src/dataloader.py).


## **3. Approche méthodologique d’exploration**

### **a. Exploration initiale des fichiers raster (.tif)**

L’analyse initiale a consisté à examiner les fichiers raster pour :
- Lire les métadonnées (dimensions, nombre de bandes, résolution).
- Visualiser les bandes individuellement :
  - Classification binaire (valeurs 0 ou 255).
  - Identification des pixels appartenant simultanément à plusieurs classes.
- Générer des histogrammes confirmant la distribution binaire des pixels.

### **b. Validation spatiale avec les fichiers vectoriels (.geojson)**

Les fichiers **GeoJSON** ont été utilisés pour valider spatialement les annotations :
- Chargement et visualisation des contours géographiques de chaque classe.
- Superposition des géométries vectorielles sur les images raster.
- Confirmation de la cohérence et de la précision des annotations.

---



## **4. Analyse de la qualité des données (PF-QA)**

L’étude des fichiers **PF-QA (Planet Fusion Quality Assurance)** a permis de :
- Identifier les pixels basés sur des observations satellites directes vs interpolées.
- Vérifier la cohérence spatiale des annotations pour garantir la fiabilité des données d’apprentissage.

---


# **Exploration des fichiers et structuration des données**


## **1. Organisation et exploration des fichiers**
Le chemin principal des données est défini par `PATH_DATA`, qui contient :
- Les images satellites **Planet Fusion** (`planet.10N/planet/10N/`).
- Les labels annotés (`labels/labels/` et `dynamicearthnet_test_labels`).

L’exploration initiale a permis :
- 📂 **D’identifier les fichiers et répertoires disponibles**.
- 🏷️ **De compter et comparer les labels d’entraînement et de test**.
- 📍 **D’afficher les sous-dossiers Planet contenant les images TIFF** pour une organisation claire des données.

---

## **2. Construction des DataFrames de labels et images**
L’organisation des labels a été systématisée en DataFrames `df_labels_train` et `df_labels_test` :
- **Extraction des clés d’identification** pour assurer la correspondance avec les images satellites.
- **Identification des labels d’entraînement et de test**.
- **Création d’un chemin structuré pour chaque label**.

Les images satellites ont été associées aux labels via une fusion avec `df_planet`, permettant de structurer les correspondances entre annotations et images sources.

---

## **3. Localisation géographique des images**
L’analyse de la **localisation des images satellites** a été effectuée en croisant leurs coordonnées avec une base de données mondiale des pays (`ne_110m_admin_0_countries.shp`).

- 🌍 **Transformation des coordonnées vers EPSG:4326** (longitude/latitude).
- 🗺️ **Détection du pays et du continent** associé à chaque image satellite.
- 🔗 **Association des informations géographiques aux métadonnées des images**.

---

## **4. Vérification de l’alignement des images satellites**
Un contrôle d’alignement des images (`ImageAlignementCheck`) a été effectué pour s’assurer de leur qualité, nous avions initialement prévu de faire avec le suivi temporelle mais nous avons ensuite abandonné cette idée :
- 📏 **Analyse des décalages moyens et maximaux entre images successives**.
- ❌ **Identification des images mal alignées** afin de les exclure du futur entraînement du modèle.

Les résultats ont été stockés dans `df_results.csv` et intégrés dans `df_merged_extended` pour une meilleure gestion des images.

---


L’ensemble de ces étapes garantit une **base de données robuste et de haute qualité** pour l'entraînement du **réseau de neurones de segmentation spatiale**. 🚀
